# 데이터 크롤링

웹 페이지는 기본 적으로 HTML 문서로 구성되어 있다.

이외에 javascript, css도 사용되나, 데이터 크롤링에서는 데이터의 내용을 다루는 것으르로 여기서는 HTML 문서를 기본으로 살펴본다.

우선 아래와 같은 라이브러리가 필요하다

In [ ]:
from bs4 import BeautifulSoup # 파이썬으로 HTML을 다루기 위한 라이브러리
import requests # 요청을 위한 라이브러리
import time # 시간을 조절하기 위한 라이브러리

크롤링할 웹페이지의 주소를 복사하여 변수에 저장한다.

(주의) 항상 메인 화면에서 무언가를 검색한 후 해당 페이지의 URL을 가져오기

네이버의 회차별 로또 번호를 제공하는 기능을 통해 로또 정보의 데이터 크롤링을 진행해보자.

로또 페이지를 가져온 URL:
https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EB%A1%9C%EB%98%90

In [ ]:
url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EB%A1%9C%EB%98%90'
res = requests.get(url)
res

<Response [200]>

웹페이지를 requests를 통해 가져왔다. 'Respense [200]'이 출력되면 정상적으로 작동한 것

In [ ]:
res.text[0:3000] # 앞에서 3000자만 출력하였다.

'<!doctype html> <html lang="ko"><head> <meta charset="utf-8"> <meta name="referrer" content="always">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta name="viewport" content="width=device-width,initial-scale=1.0,maximum-scale=2.0"> <meta property="og:title" content="로또 : 네이버 통합검색"/> <meta property="og:image" content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png"> <meta property="og:description" content="\'로또\'의 네이버 통합검색 결과입니다."> <meta name="description" lang="ko" content="\'로또\'의 네이버 통합검색 결과입니다."> <title>로또 : 네이버 통합검색</title> <link rel="shortcut icon" href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_191118_pc.ico">  <link rel="search" type="application/opensearchdescription+xml" href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" title="Naver" /><script> if (top.frames.length!=0 || window!=top) window.open(location, "_top"); </script><link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/s

위와 같이 text를 불려오면 웹 페이지의 html을 가져온다. 하지만 출력결과를 잘 보면 진짜 html 문서가 아닌 양쪽에 '가 있는 text 문서이다. 이를 진짜 html 문서로 변환해보자

In [ ]:
html = BeautifulSoup(res.text)
# html # html을 출력해 보면 문자열이 아닌 html 형식대로 출력된다

## 원하는 정보 찾기

### 로또 회차 정보 찾기

웹 페이지에서 로또 회차가 나와았는 부분에서 우클릭 후 '검사'를 클릭한다.
페이지 소스에서 로또 회차 정보가 a tag 안에 들어있는 것을 확인한다.

``` html
<a onclick="return goOtherCR(this,&quot;a=nco_x5e*2&amp;r=1&amp;i=&quot;+urlencode(&quot;0011AD9E_000001E7C267&quot;)+&quot;&amp;u=&quot;+urlencode(this.href));" class="text _select_trigger _text" href="" aria-expanded="false">1078회차 (2023.07.29.)</a>
```

a 라는 tag 만으로 원하는 위치를 찾기에는 페이지의 a 태그가 너무 많다.

In [6]:
current = html.find('a')
current

<a href="#lnb"><span>메뉴 영역으로 바로가기</span></a>

위와 같이 원하는 위치의 값이 아니다.

좀 더 구체적으로 찾기 위해서 class 이름을 알려주자

In [16]:
current = html.find('a', class_="text _select_trigger _text")
current

<a aria-expanded="false" class="text _select_trigger _text" href="" onclick='return goOtherCR(this,"a=nco_x5e*2&amp;r=1&amp;i="+urlencode("0011AD9E_000001E7C267")+"&amp;u="+urlencode(this.href));'>1078회차 (2023.07.29.)</a>

class에 _를 붙여 위와 같이 검색하면 회차 정보가 담긴 원하는 위치의 tag를 검색이 가능하다.

find() 함수를 사용하여 찾은 결과도 html이기 때문에 tag 안에 tag를 찾는다면 find 함수를 연쇄적으로 사용하여 검색이 가능하다.

```
current = html.find(---).find(---)
```

찾은 current는 tag와 text로 이루어져 있다. 원하는 정보는 text 정보이므로 text만 추출해보자

In [9]:
current.text

'1078회차 (2023.07.29.)'

text를 이용해 추출한 결과는 더이상 html이 아니라 문자열이다. 문자열 함수를 이용하여 회차 정보 중 숫자만 추출할 수 있다.

In [17]:
current = int(current.text.split(' ')[0].replace('회차',''))
current

1078

- 공백을 기준으로 문자열을 나눈 후
- 만들어진 list에서 앞부분을 추출
- '회차' 문자를 replace 함수를 제거한다.
- 정수형으로 변환한다.

### 로또 번호 찾기

In [24]:
numbers = html.find('div',class_="win_ball").text.split()
numbers

['6', '10', '11', '14', '36', '38', '43']

리스트 안에 문자열이 담겨있는 형태로 저장되어 있으므로 각 요소를 map() 함수를 이용하여 정수형으로 변환하자

In [28]:
numbers = list(map(int, numbers))
numbers

[6, 10, 11, 14, 36, 38, 43]

map 함수를 통해 각 변수를 int 형으로 변환하고, 이 값들을 다시 list로 변환한다.

### 모든 회차의 정보 저장하기